# 뉴스 카테고리 분류하기

뉴스 기사 본문을 크롤링하여 가져 왔다면, 기사 본문 텍스트 데이터를 임베딩할 차례입니다.  
형태소 단위로 임베딩할 것이기 때문에 전처리 과정이 필요합니다.    
전처리 과정이 끝나면 doc2vec을 통해 코퍼스를 임베딩 하고, 그 후에 분류를 진행합니다. 

> 순서
1. 기사 본문 토큰화& POS 태깅& 의미있는 형태소만 추출
2. 임베딩 : word2vec과 doc2vec을 적용해본다.   
3. Doc2vec 모델 훈련시키기 - DM,DBOW
4. 가장 성능이 좋은 Doc2Vec 모델 선택
5. 임베딩 결과와 tensorflow를 활용하여 분류

In [1]:
import konlpy
import csv
import pandas as pd
from pandas import DataFrame as df
import numpy as np
from pprint import pprint
from konlpy.tag import Twitter
twitter = Twitter()
from gensim.models import doc2vec
from gensim.models.word2vec import Word2Vec
import pickle

## Data import
#### 네이버 뉴스기사 데이터 불러오기
R로 text 파일 합치고, 카테고리 100~105 에서 0~5 로 바꾸었습니다. ( NewsData.csv 파일)   
> #### 변수 설명
* ID : 0~29 , 해당 카테고리 내에서 기사 ID
* Category : 0-5, 총 6개 카테고리   
    정치(0), 경제(1), 사회(2), 생활/문화(3), 세계(4), IT/과학(5)
* Week : 2017년 1월 첫째 주(106) ~ 2018년 1월 셋째 주(119) / **55 Weeks / 9,696 개 기사**    
    (17년 1~5월은 카테고리당 30개가 아니라 29개입니다.)
* Title : 기사 제목
* Contents : 기사 본문

In [2]:
def read_comm_csv(filename):
    data_table = []
    with open(filename, 'r',encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
            data_table.append(row)
    return data_table

In [3]:
data = read_comm_csv('newsData_55weeks.csv')

In [4]:
df(data[1:], columns=data[0]).head() # 1열(head) 제외

,ID,Category,Week,Title,Contents
0,0,0,106,"[단독]정유라, “(주사 아줌마)누구인지 알 것 같다”…현지 답변태도 분석, 사전 ...",덴마크 올보르 법원에서 잠시 휴정중 기자들의 질문에 답변하는 정유라씨\n\n사진=현...
1,1,0,106,"[단독]""""정유라, 이대학장실 등 교내서 교수 6명에 학점취득 코치받아""""",[연합뉴스TV제공] [연합뉴스TV제공]\n\n제4차청문회에 출석한 김경숙 전 이화여...
2,2,0,106,[단독 동영상] 정유라 덴마크 법원 진술 공개,"동영상 뉴스\n\n정유라 / 최순실씨 딸""""차은택씨도 전 딱 한번 봤어요. 그 테스..."
3,3,0,106,"정유라, 덴마크서 불법 체류 혐의로 체포···특검 “송환 협조중” (종합)",[아시아경제 정준영 기자] 이화여대 학사부정 및 삼성 특혜지원 의혹의 수혜자 겸 공...
4,4,0,106,"[단독 영상] 정유라, 덴마크 현지서 체포…국내 송환 임박",동영상 뉴스\n\n[앵커]그럼 정유라씨 체포 소식부터 알아봅니다.덴마크 현지에서 이...


In [5]:
len(data) # 총 기사 수  + head 

9697

## 1) Word2Vec 적용하기
word2vec에는 문맥을 통해 단어를 예측하는 CBOW, 단어를 통해 문맥을 예측하는 Skip-Gram 두가지 기법이 있습니다.    
아래 예시를 통해 차이점을 살펴보도록 하겠습니다.   


>   *** " When I find myself in times of trouble mother Mary comes to me."* **     
   - CBOW Input : [When, I, find], [I, find, myself], [myself, in, times], ...    
   한 문장에서 **연속적인 단어들**을 입력값으로 집어 넣는다.   
   - Skip-Gram Input : [When, I, find], [When, I, myself], [When, find, myself], ...      
   한 문장 안에서 **발생할 수 있는 모든 단어 모음(Bag of words)**를 입력값으로 취한다.   
   
**CBOW**는 Skip-Gram보다 입력값이 적기 때문에 학습 시간이 더 짧고, 자주 등장하는 단어에 대해 정확한 결과를 도출해낼 수 있습니다.    
**Skip-Gram**은 학습 시간이 더 길지만 작은 데이터셋에 대해 효율적이고, 자주 등장하지 않는 단어에 대해 더 높은 정확도를 갖습니다.

## 1-1. 데이터 토큰화
KoNPLy의 Twitter 사용하여 tokenize 및 POS tagging 진행하였습니다.

In [6]:
def tokenize(doc) : 
    return ['/'.join(t) for t in twitter.pos(doc, norm=True, stem=True)]

tokens_doc = [ tokenize(row[4]) for row in data[1:] ] # head 제외

In [7]:
df(tokens_doc).head()

,0,1,2,3,4,5,6,7,8,9,...,13230,13231,13232,13233,13234,13235,13236,13237,13238,13239
0,덴마크/Noun,올보르/Noun,법원/Noun,에서/Josa,잠시/Noun,휴정/Noun,중/Suffix,기자/Noun,들/Suffix,의/Josa,...,None,None,None,None,None,None,None,None,None,None
1,[/Punctuation,연합뉴스/Noun,TV/Alpha,제공/Noun,]/Punctuation,[/Punctuation,연합뉴스/Noun,TV/Alpha,제공/Noun,]/Punctuation,...,None,None,None,None,None,None,None,None,None,None
2,동영상/Noun,뉴스/Noun,정유/Noun,라/Josa,//Punctuation,최순실/Noun,씨/Suffix,딸/Noun,"""""/Punctuation",차/Noun,...,None,None,None,None,None,None,None,None,None,None
3,[/Punctuation,아시아/Noun,경제/Noun,정준영/Noun,기자/Noun,]/Punctuation,이화여대/Noun,학사/Noun,부정/Noun,및/Noun,...,None,None,None,None,None,None,None,None,None,None
4,동영상/Noun,뉴스/Noun,[/Punctuation,앵커/Noun,]/Punctuation,그렇다/Adjective,정유/Noun,라씨/Noun,체포/Noun,소식/Noun,...,None,None,None,None,None,None,None,None,None,None


## 1-2. 품사 필터링
의미가 있는 형태소만 (명사, 동사, 형용사, 영어) 남깁니다.

In [8]:
def pumsa(doc):
    temp = []
    
    for items in doc:
        words = []
        
        for item in items:
            if item.split('/')[1] in ['Noun', 'Verb', 'Adjective','Alpha']:
                words.append(item)             
        temp.append(words)
    return temp

In [9]:
selected_tokens = pumsa(tokens_doc)
df(selected_tokens).head()

,0,1,2,3,4,5,6,7,8,9,...,8065,8066,8067,8068,8069,8070,8071,8072,8073,8074
0,덴마크/Noun,올보르/Noun,법원/Noun,잠시/Noun,휴정/Noun,기자/Noun,질문/Noun,답변/Noun,하다/Verb,정유/Noun,...,None,None,None,None,None,None,None,None,None,None
1,연합뉴스/Noun,TV/Alpha,제공/Noun,연합뉴스/Noun,TV/Alpha,제공/Noun,제/Noun,차/Noun,청문회/Noun,출석/Noun,...,None,None,None,None,None,None,None,None,None,None
2,동영상/Noun,뉴스/Noun,정유/Noun,최순실/Noun,딸/Noun,차/Noun,은택/Noun,전/Noun,한번/Noun,보다/Verb,...,None,None,None,None,None,None,None,None,None,None
3,아시아/Noun,경제/Noun,정준영/Noun,기자/Noun,이화여대/Noun,학사/Noun,부정/Noun,및/Noun,삼성/Noun,특혜/Noun,...,None,None,None,None,None,None,None,None,None,None
4,동영상/Noun,뉴스/Noun,앵커/Noun,그렇다/Adjective,정유/Noun,라씨/Noun,체포/Noun,소식/Noun,알아보다/Verb,덴마크/Noun,...,None,None,None,None,None,None,None,None,None,None


#### 각 기사마다 필터링된 토큰의 개수를 세어보겠습니다.

In [10]:
# 각 기사별 토큰 개수
a=df(selected_tokens).loc[:,:].count(axis=1)
print(a[0:5])

0    741
1    482
2    175
3    421
4    197
dtype: int64


In [11]:
max(a)

8075

In [12]:
# 총 토큰 개수
sum(a)

4897656

## 1-3. 임베딩 

파이썬에서는 gensim이라는 패키지의 Word2Vec 클래스로 구현가능 합니다.   
논문에 의하면 Skip-gram이 CBOW에 비해 전반적으로 좋은 결과를 내는 것으로 보여 Skip-gram 모델을 사용하도록 하겠습니다.

> * sg=1,  # By default (sg=0), CBOW is used. Otherwise (sg=1), skip-gram is employed.   
* size=100,  # the dimensionality of the feature vectors.  
* window=10, # the maximum distance between the current and predicted word within a sentence. 
* min_count=5   #ignore all words with total frequency lower than this.

In [13]:
w2v_model= Word2Vec(sentences=selected_tokens, sample=1e-5, sg=1,size=100,window=10, min_count=5)

* 단어간 유사도 확인(cosine-similarity)

In [14]:
w2v_model.init_sims(replace=True)

In [15]:
w2v_model.similarity('아이폰/Noun', '애플/Noun')

/home/hufs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.8628591429175143

* 유사도 높은 순으로 10개 단어 추출

In [16]:
w2v_model.most_similar("평창/Noun")

/home/hufs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('동계올림픽/Noun', 0.7810097932815552),
 ('올림픽/Noun', 0.7472667098045349),
 ('평창올림픽/Noun', 0.7230064868927002),
 ('패럴림픽/Noun', 0.7062085866928101),
 ('알펜시아/Noun', 0.6842470169067383),
 ('국제올림픽위원회/Noun', 0.6815958023071289),
 ('아이스하키/Noun', 0.6760965585708618),
 ('평창동계올림픽/Noun', 0.6716214418411255),
 ('정선/Noun', 0.6632678508758545),
 ('평창군/Noun', 0.6616650819778442)]

In [17]:
w2v_model.most_similar("김정은/Noun")

/home/hufs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('노동당/Noun', 0.8926966190338135),
 ('노동신문/Noun', 0.8606791496276855),
 ('김일성/Noun', 0.8475520610809326),
 ('조선노동당/Noun', 0.8314770460128784),
 ('Redistribution/Alpha', 0.8139813542366028),
 ('양강도/Noun', 0.8073557019233704),
 ('김효정/Noun', 0.7967101335525513),
 ('리병철/Noun', 0.791688084602356),
 ('북한/Noun', 0.7897021174430847),
 ('태양절/Noun', 0.7847779393196106)]

In [18]:
w2v_model.most_similar("한파/Noun")

/home/hufs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('체감온도/Noun', 0.8757126331329346),
 ('맹위/Noun', 0.8734347820281982),
 ('선선/Noun', 0.8670997619628906),
 ('한낮/Noun', 0.8637862205505371),
 ('이맘/Noun', 0.863343358039856),
 ('눈비/Noun', 0.8612349629402161),
 ('천둥/Noun', 0.8570339679718018),
 ('만조/Noun', 0.8540732860565186),
 ('영서지방/Noun', 0.8526050448417664),
 ('꽁꽁/Noun', 0.8522484302520752)]

In [19]:
w2v_model.most_similar("비트코인/Noun")

/home/hufs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('가상화/Noun', 0.8640475273132324),
 ('채굴/Noun', 0.857657790184021),
 ('코인/Noun', 0.8241461515426636),
 ('리플/Noun', 0.8213384747505188),
 ('캐시/Noun', 0.8042333722114563),
 ('열풍/Noun', 0.7987518906593323),
 ('화폐/Noun', 0.7961705327033997),
 ('가상/Noun', 0.791966438293457),
 ('폐인/Noun', 0.7892058491706848),
 ('암호/Noun', 0.7793246507644653)]

--> word2vec을 활용한 임베딩이 잘 된 것을 확인할 수 있습니다.

# 2) Doc2vec 적용하기

## 2-1. preprocessing
Word2vec 과 동일하게 데이터 토큰화 + 품사 필터링 진행합니다. (단, 품사필터링함수를 리스트로 반환하도록 수정하였습니다)    

In [20]:
def pumsa_2(doc):
    temp = []
    
    for items in doc:
        words = []
        
        for item in items:
            if item.split('/')[1] in ['Noun', 'Verb', 'Adjective','Alpha']:
                words.append(item)             
        temp.append([words])  #  각 row를 리스트로 반환
    return temp

selected_tokens = pumsa_2(tokens_doc)

In [21]:
df(selected_tokens).tail()

,0
9691,"[최근/Noun, 애플스토어/Noun, 몰려들다/Verb, 고객/Noun, 때문/N..."
9692,"[CJ/Alpha, 그룹/Noun, BIG/Alpha, PICTURE/Alpha, ..."
9693,"[년/Noun, 일자리/Noun, 만/Noun, 창/Noun, 추다/Verb, 등/..."
9694,"[아이폰/Noun, 아이폰/Noun, 머니투데이/Noun, 서진욱/Noun, 기자/..."
9695,"[글꼴/Noun, 선택/Noun, 본문/Noun, 텍스트/Noun, 작다/Adjec..."


In [22]:
tot_data = np.append(data[1:], selected_tokens, 1)

In [23]:
df(tot_data).head()

,0,1,2,3,4,5
0,0,0,106,"[단독]정유라, “(주사 아줌마)누구인지 알 것 같다”…현지 답변태도 분석, 사전 ...",덴마크 올보르 법원에서 잠시 휴정중 기자들의 질문에 답변하는 정유라씨\n\n사진=현...,"[덴마크/Noun, 올보르/Noun, 법원/Noun, 잠시/Noun, 휴정/Noun..."
1,1,0,106,"[단독]""""정유라, 이대학장실 등 교내서 교수 6명에 학점취득 코치받아""""",[연합뉴스TV제공] [연합뉴스TV제공]\n\n제4차청문회에 출석한 김경숙 전 이화여...,"[연합뉴스/Noun, TV/Alpha, 제공/Noun, 연합뉴스/Noun, TV/A..."
2,2,0,106,[단독 동영상] 정유라 덴마크 법원 진술 공개,"동영상 뉴스\n\n정유라 / 최순실씨 딸""""차은택씨도 전 딱 한번 봤어요. 그 테스...","[동영상/Noun, 뉴스/Noun, 정유/Noun, 최순실/Noun, 딸/Noun,..."
3,3,0,106,"정유라, 덴마크서 불법 체류 혐의로 체포···특검 “송환 협조중” (종합)",[아시아경제 정준영 기자] 이화여대 학사부정 및 삼성 특혜지원 의혹의 수혜자 겸 공...,"[아시아/Noun, 경제/Noun, 정준영/Noun, 기자/Noun, 이화여대/No..."
4,4,0,106,"[단독 영상] 정유라, 덴마크 현지서 체포…국내 송환 임박",동영상 뉴스\n\n[앵커]그럼 정유라씨 체포 소식부터 알아봅니다.덴마크 현지에서 이...,"[동영상/Noun, 뉴스/Noun, 앵커/Noun, 그렇다/Adjective, 정유..."


#### doc2vec이 요구하는 형태로 데이터 변환
* ( 본문 , 카테고리) 형태로 변환합니다

In [24]:
tot_docs = [(row[5],row[1]) for row in tot_data]
df(tot_docs).tail()

,0,1
9691,"[최근/Noun, 애플스토어/Noun, 몰려들다/Verb, 고객/Noun, 때문/N...",5
9692,"[CJ/Alpha, 그룹/Noun, BIG/Alpha, PICTURE/Alpha, ...",5
9693,"[년/Noun, 일자리/Noun, 만/Noun, 창/Noun, 추다/Verb, 등/...",5
9694,"[아이폰/Noun, 아이폰/Noun, 머니투데이/Noun, 서진욱/Noun, 기자/...",5
9695,"[글꼴/Noun, 선택/Noun, 본문/Noun, 텍스트/Noun, 작다/Adjec...",5


#### 데이터(train, test 8:2) 나누기 
* 카테고리별로 층화 추출(stratified splitting)   
class(뉴스 카테고리) 별로 고르게 뽑기 위하여 strtify 옵션을 추가합니다 

In [25]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(tot_docs, stratify =df(tot_docs).iloc[:,1] , test_size=0.2) 
# df(tot_docs).iloc[:,1] :  2열(Category)기준으로 층화

* 각 카테고리마다 동일한 개수의 기사가 샘플링된 것을 확인할 수 있습니다.

In [26]:
sum(df(train).iloc[:,1]=='0') # train set에서 '정치' 기사 갯수

1293

In [27]:
sum(df(train).iloc[:,1]=='5')  # train set에서 'IT/과학' 기사 갯수

1292

In [28]:
len(train) # 전체 train set 기사 수

7756

In [29]:
len(test) # 전체 test set 기사 수

1940

* 텍스트 파일로 저장해줍니다.

In [30]:
with open('train.txt', 'wb') as file:
    pickle.dump(train,file)
    
with open('test.txt', 'wb') as file:
    pickle.dump(test,file)

#### Naver_news_classification_part2 에서 계속 >